### 导包

In [84]:
!pip install toad

In [85]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import toad

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,roc_auc_score, auc

import xgboost as xgb
from xgboost.sklearn import XGBClassifier, XGBRegressor
import lightgbm as lgb

### 原数据读取

#### 原始数据

In [86]:
def data_pred():
    df_init=pd.read_csv('/kaggle/input/risk-control/risk.csv')
    df=df_init.sample(50000).reset_index(drop=True)
    
    df['y']=df['loan_status'].map({'Fully Paid':0,'Late (31-120 days)':1,'Late (16-30 days)':1})
    df.drop(columns=['loan_status'], inplace=True)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    df.drop(columns=['id'], inplace=True)
    df.drop(columns=['member_id'], inplace=True)
    
    df_copy=df[df['y'].isin([0,1])].copy()
    return df_copy.reset_index(drop=True)

df=data_pred()
df.shape

(24292, 149)

In [87]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,y
0,6000.0,6000.0,6000.0,36 months,12.39,200.41,C,C1,Accounting Office Manager,10+ years,MORTGAGE,66000.0,Not Verified,Dec-2014,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,028xx,RI,26.49,1.0,Oct-1999,675.0,679.0,0.0,5.0,NaN,9.0,0.0,23576.0,93.2,32.0,f,0.0,0.0,7134.314182,7134.31,6000.0,1134.31,0.0,0.0,0.0,Mar-2017,1931.91,NaN,Dec-2018,649.0,645.0,0.0,47.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,494727.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25300.0,NaN,NaN,NaN,6.0,54970.0,57.0,99.6,0.0,0.0,106.0,182.0,9.0,7.0,4.0,13.0,NaN,7.0,5.0,1.0,3.0,5.0,3.0,13.0,8.0,5.0,20.0,5.0,9.0,0.0,0.0,0.0,2.0,93.7,100.0,0.0,0.0,542288.0,45904.0,16200.0,32687.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,6000.0,6000.0,6000.0,36 months,19.05,220.09,D,D4,Goodwill,10+ years,MORTGAGE,69680.0,Not Verified,Nov-2012,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Loan,992xx,WA,13.28,0.0,Jan-2004,665.0,669.0,0.0,NaN,NaN,24.0,0.0,13389.0,88.1,37.0,f,0.0,0.0,7438.338303,7438.34,6000.0,1438.34,0.0,0.0,0.0,Jul-2014,3483.51,NaN,Jan-2015,659.0,655.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,180964.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15200.0,NaN,NaN,NaN,6.0,7540.0,1112.0,91.2,0.0,0.0,108.0,99.0,9.0,3.0,3.0,26.0,NaN,7.0,NaN,0.0,6.0,8.0,6.0,7.0,24.0,8.0,9.0,8.0,24.0,0.0,0.0,0.0,4.0,100.0,83.3,0.0,0.0,185279.0,77500.0,12700.0,62287.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [88]:
df.select_dtypes(include=['string','object']).head()

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
0,36 months,C,C1,Accounting Office Manager,10+ years,MORTGAGE,Not Verified,Dec-2014,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,028xx,RI,Oct-1999,f,Mar-2017,NaN,Dec-2018,Individual,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
1,36 months,D,D4,Goodwill,10+ years,MORTGAGE,Not Verified,Nov-2012,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Loan,992xx,WA,Jan-2004,f,Jul-2014,NaN,Jan-2015,Individual,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
2,60 months,C,C5,Drafter/Engineer,3 years,RENT,Source Verified,Oct-2015,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,891xx,NV,Sep-2002,w,Jan-2017,NaN,Jan-2017,Individual,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
3,60 months,B,B4,Software Engineer,3 years,MORTGAGE,Source Verified,Dec-2013,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 11/29/13 > Looking to cons...,debt_consolidation,Debt Consolidation,532xx,WI,Oct-2004,w,Sep-2017,NaN,Sep-2017,Individual,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN
4,36 months,A,A2,NaN,NaN,MORTGAGE,Verified,Sep-2014,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,852xx,AZ,Feb-1989,f,Sep-2017,NaN,Sep-2017,Individual,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN


In [89]:
# 日期特征
date_fea=[
    'issue_d',
    'earliest_cr_line',
    'last_pymnt_d',
    'last_credit_pull_d',
    'sec_app_earliest_cr_line',
    'hardship_start_date',
    'hardship_end_date',
    'payment_plan_start_date',
    'debt_settlement_flag_date',
    'settlement_date',
    'next_pymnt_d'
]

# 类别特征
target_label_fea=[
    'sub_grade',
    'emp_title',
    'home_ownership',
    'verification_status',
    'pymnt_plan',
    'purpose',
    'addr_state',
    'initial_list_status',
    'application_type',
    'verification_status_joint',
    'hardship_flag',
    'hardship_type',
    'hardship_reason',
    'hardship_status',
    'hardship_loan_status',
    'disbursement_method',
    'debt_settlement_flag',
    'settlement_status',        
]

In [90]:
df['y'].value_counts()

y
0.0    23716
1.0      576
Name: count, dtype: int64

#### 查看数据

In [91]:
for i in df.columns[0:10]:
    print(str(i), df[i][0])

loan_amnt 6000.0
funded_amnt 6000.0
funded_amnt_inv 6000.0
term  36 months
int_rate 12.39
installment 200.41
grade C
sub_grade C1
emp_title Accounting Office Manager
emp_length 10+ years


In [92]:
for i in df.columns[10:20]:
    print(str(i), df[i][0])

home_ownership MORTGAGE
annual_inc 66000.0
verification_status Not Verified
issue_d Dec-2014
pymnt_plan n
url https://lendingclub.com/browse/loanDetail.action?loan_id=37811804
desc nan
purpose debt_consolidation
title Debt consolidation
zip_code 028xx


In [93]:
for i in df.columns[20:30]:
    print(str(i), df[i][0])

addr_state RI
dti 26.49
delinq_2yrs 1.0
earliest_cr_line Oct-1999
fico_range_low 675.0
fico_range_high 679.0
inq_last_6mths 0.0
mths_since_last_delinq 5.0
mths_since_last_record nan
open_acc 9.0


In [94]:
for i in df.columns[30:40]:
    print(str(i), df[i][0])

pub_rec 0.0
revol_bal 23576.0
revol_util 93.2
total_acc 32.0
initial_list_status f
out_prncp 0.0
out_prncp_inv 0.0
total_pymnt 7134.314182275501
total_pymnt_inv 7134.31
total_rec_prncp 6000.0


In [95]:
for i in df.columns[40:50]:
    print(str(i), df[i][0])

total_rec_int 1134.31
total_rec_late_fee 0.0
recoveries 0.0
collection_recovery_fee 0.0
last_pymnt_d Mar-2017
last_pymnt_amnt 1931.91
next_pymnt_d nan
last_credit_pull_d Dec-2018
last_fico_range_high 649.0
last_fico_range_low 645.0


In [96]:
for i in df.columns[50:60]:
    print(str(i), df[i][0])

collections_12_mths_ex_med 0.0
mths_since_last_major_derog 47.0
policy_code 1.0
application_type Individual
annual_inc_joint nan
dti_joint nan
verification_status_joint nan
acc_now_delinq 0.0
tot_coll_amt 0.0
tot_cur_bal 494727.0


In [97]:
for i in df.columns[60:70]:
    print(str(i), df[i][0])

open_acc_6m nan
open_act_il nan
open_il_12m nan
open_il_24m nan
mths_since_rcnt_il nan
total_bal_il nan
il_util nan
open_rv_12m nan
open_rv_24m nan
max_bal_bc nan


In [98]:
for i in df.columns[70:80]:
    print(str(i), df[i][0])

all_util nan
total_rev_hi_lim 25300.0
inq_fi nan
total_cu_tl nan
inq_last_12m nan
acc_open_past_24mths 6.0
avg_cur_bal 54970.0
bc_open_to_buy 57.0
bc_util 99.6
chargeoff_within_12_mths 0.0


In [99]:
for i in df.columns[80:90]:
    print(str(i), df[i][0])

delinq_amnt 0.0
mo_sin_old_il_acct 106.0
mo_sin_old_rev_tl_op 182.0
mo_sin_rcnt_rev_tl_op 9.0
mo_sin_rcnt_tl 7.0
mort_acc 4.0
mths_since_recent_bc 13.0
mths_since_recent_bc_dlq nan
mths_since_recent_inq 7.0
mths_since_recent_revol_delinq 5.0


In [100]:
for i in df.columns[90:100]:
    print(str(i), df[i][0])

num_accts_ever_120_pd 1.0
num_actv_bc_tl 3.0
num_actv_rev_tl 5.0
num_bc_sats 3.0
num_bc_tl 13.0
num_il_tl 8.0
num_op_rev_tl 5.0
num_rev_accts 20.0
num_rev_tl_bal_gt_0 5.0
num_sats 9.0


In [101]:
for i in df.columns[100:110]:
    print(str(i), df[i][0])

num_tl_120dpd_2m 0.0
num_tl_30dpd 0.0
num_tl_90g_dpd_24m 0.0
num_tl_op_past_12m 2.0
pct_tl_nvr_dlq 93.7
percent_bc_gt_75 100.0
pub_rec_bankruptcies 0.0
tax_liens 0.0
tot_hi_cred_lim 542288.0
total_bal_ex_mort 45904.0


In [102]:
for i in df.columns[110:120]:
    print(str(i), df[i][0])

total_bc_limit 16200.0
total_il_high_credit_limit 32687.0
revol_bal_joint nan
sec_app_fico_range_low nan
sec_app_fico_range_high nan
sec_app_earliest_cr_line nan
sec_app_inq_last_6mths nan
sec_app_mort_acc nan
sec_app_open_acc nan
sec_app_revol_util nan


In [103]:
for i in df.columns[120:130]:
    print(str(i), df[i][0])

sec_app_open_act_il nan
sec_app_num_rev_accts nan
sec_app_chargeoff_within_12_mths nan
sec_app_collections_12_mths_ex_med nan
sec_app_mths_since_last_major_derog nan
hardship_flag N
hardship_type nan
hardship_reason nan
hardship_status nan
deferral_term nan


In [104]:
for i in df.columns[130:140]:
    print(str(i), df[i][0])

hardship_amount nan
hardship_start_date nan
hardship_end_date nan
payment_plan_start_date nan
hardship_length nan
hardship_dpd nan
hardship_loan_status nan
orig_projected_additional_accrued_interest nan
hardship_payoff_balance_amount nan
hardship_last_payment_amount nan


In [105]:
for i in df.columns[140:]:
    print(str(i), df[i][0])

disbursement_method Cash
debt_settlement_flag N
debt_settlement_flag_date nan
settlement_status nan
settlement_date nan
settlement_amount nan
settlement_percentage nan
settlement_term nan
y 0.0


#### 返回

In [106]:
df.shape

(24292, 149)

In [107]:
df.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,y
0,6000.0,6000.0,6000.0,36 months,12.39,200.41,C,C1,Accounting Office Manager,10+ years,MORTGAGE,66000.0,Not Verified,Dec-2014,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,028xx,RI,26.49,1.0,Oct-1999,675.0,679.0,0.0,5.0,NaN,9.0,0.0,23576.0,93.2,32.0,f,0.0,0.0,7134.314182,7134.31,6000.0,1134.31,0.0,0.0,0.0,Mar-2017,1931.91,NaN,Dec-2018,649.0,645.0,0.0,47.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,494727.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25300.0,NaN,NaN,NaN,6.0,54970.0,57.0,99.6,0.0,0.0,106.0,182.0,9.0,7.0,4.0,13.0,NaN,7.0,5.0,1.0,3.0,5.0,3.0,13.0,8.0,5.0,20.0,5.0,9.0,0.0,0.0,0.0,2.0,93.7,100.0,0.0,0.0,542288.0,45904.0,16200.0,32687.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,6000.0,6000.0,6000.0,36 months,19.05,220.09,D,D4,Goodwill,10+ years,MORTGAGE,69680.0,Not Verified,Nov-2012,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Loan,992xx,WA,13.28,0.0,Jan-2004,665.0,669.0,0.0,NaN,NaN,24.0,0.0,13389.0,88.1,37.0,f,0.0,0.0,7438.338303,7438.34,6000.0,1438.34,0.0,0.0,0.0,Jul-2014,3483.51,NaN,Jan-2015,659.0,655.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,180964.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15200.0,NaN,NaN,NaN,6.0,7540.0,1112.0,91.2,0.0,0.0,108.0,99.0,9.0,3.0,3.0,26.0,NaN,7.0,NaN,0.0,6.0,8.0,6.0,7.0,24.0,8.0,9.0,8.0,24.0,0.0,0.0,0.0,4.0,100.0,83.3,0.0,0.0,185279.0,77500.0,12700.0,62287.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [108]:
# 单变量目标编码，返回字典 col: y_mean
def target_label_mean(df_train, col, y_name='y'): 
    return dict(round(df_train.groupby([col])[y_name].mean(), 6))  

# 全部变量目标编码
def target_label_mean_all(df_train, col_list, y_name='y'): 
    return {col: target_label_mean(df_train, col, y_name) for col in col_list}

 # df 目标编码
def train_test_target_label(df_copy, col_list): 
    df=df_copy.pipe(train_test_split_self)
    all_col_map=target_label_mean_all(df[df.sample_label=='train'], 
                                      col_list, y_name='y')
    for col in col_list:
        df[col]=df[col].map(all_col_map[col])
    
    return df,all_col_map

# 缺失填充，均值填充
def get_fill_df(df_pred):
    df=df_pred.copy()
    fill_col=[col for col in df.columns if col not in ['y', 'sample_label']]
    for i in fill_col:
        df[i]=df[i].fillna(df[i].mean())
    return df

# 数据划分, sample_labels
def train_test_split_self(df_copy, y_name='y'):  
    frac=0.7
    train_index=list(df_copy[df_copy[y_name]==1].sample(frac=frac, random_state=1).index)+list(df_copy[df_copy[y_name]==0].sample(frac=frac,random_state=1).index)
    df_copy.loc[df_copy.index.isin(train_index),'sample_label']='train'
    df_copy.loc[~df_copy.index.isin(train_index),'sample_label']='test'
    return df_copy

# 所有预处理汇总
def fea_engineer(df_copy):  
    
    df=df_copy.copy()
    
    # term
    df.term=df.term.str.replace('months','').str.replace(' ','').astype('float')
    
    # grade
    grade_map=dict(enumerate(list('ABCDEFG')))
    grade_map={value:key for key,value in grade_map.items()}
    df.grade=df.grade.map(grade_map)
    
    # emp_length
    df.emp_length=df.emp_length.str.replace('years','').str.replace('year','').str.replace(' ','').map({'<1':0,'10+':10})
    
    # float_col
    float_col=list(df_copy.select_dtypes(include=['int','float']).columns)
    float_col.remove('y')
    
    # drop col + drop date 
    drop_col=['url','desc','title','zip_code']
    df=df.drop(drop_col+date_fea, axis=1)  # 日期字段含义不明确，所以先不用了
    
    # 部分类别型变量进行 target_label 编码
    df, all_col_map=train_test_target_label(df, target_label_fea)  
    return df, all_col_map

df_pred, all_col_map=fea_engineer(df)
df_pred.shape

(24292, 135)

In [109]:
df_pred.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,settlement_status,settlement_amount,settlement_percentage,settlement_term,y,sample_label
0,6000.0,6000.0,6000.0,36.0,12.39,200.41,2,0.016729,0.000000,10.0,0.021291,66000.0,0.021171,0.023298,0.024599,0.025316,26.49,1.0,675.0,679.0,0.0,5.0,NaN,9.0,0.0,23576.0,93.2,32.0,0.011226,0.0,0.0,7134.314182,7134.31,6000.0,1134.31,0.0,0.0,0.0,1931.91,649.0,645.0,0.0,47.0,1.0,0.021163,NaN,NaN,NaN,0.0,0.0,494727.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25300.0,NaN,NaN,NaN,6.0,54970.0,57.0,99.6,0.0,0.0,106.0,182.0,9.0,7.0,4.0,13.0,NaN,7.0,5.0,1.0,3.0,5.0,3.0,13.0,8.0,5.0,20.0,5.0,9.0,0.0,0.0,0.0,2.0,93.7,100.0,0.0,0.0,542288.0,45904.0,16200.0,32687.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023406,0.022838,NaN,NaN,NaN,NaN,0.0,train
1,6000.0,6000.0,6000.0,36.0,19.05,220.09,3,0.016129,NaN,10.0,0.021291,69680.0,0.021171,0.023298,0.024599,0.021951,13.28,0.0,665.0,669.0,0.0,NaN,NaN,24.0,0.0,13389.0,88.1,37.0,0.011226,0.0,0.0,7438.338303,7438.34,6000.0,1438.34,0.0,0.0,0.0,3483.51,659.0,655.0,0.0,NaN,1.0,0.021163,NaN,NaN,NaN,0.0,0.0,180964.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15200.0,NaN,NaN,NaN,6.0,7540.0,1112.0,91.2,0.0,0.0,108.0,99.0,9.0,3.0,3.0,26.0,NaN,7.0,NaN,0.0,6.0,8.0,6.0,7.0,24.0,8.0,9.0,8.0,24.0,0.0,0.0,0.0,4.0,100.0,83.3,0.0,0.0,185279.0,77500.0,12700.0,62287.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023406,0.022838,NaN,NaN,NaN,NaN,0.0,test
2,20000.0,20000.0,20000.0,60.0,14.65,472.14,2,0.042998,0.000000,NaN,0.025533,52000.0,0.024966,0.023298,0.024599,0.015625,17.38,0.0,660.0,664.0,0.0,45.0,NaN,6.0,0.0,6906.0,57.1,30.0,0.032699,0.0,0.0,23298.237718,23298.24,20000.0,3298.24,0.0,0.0,0.0,17162.39,719.0,715.0,0.0,59.0,1.0,0.021163,NaN,NaN,NaN,0.0,7506.0,47089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12100.0,NaN,NaN,NaN,5.0,7848.0,4594.0,60.1

In [110]:
df_pred['sample_label'].value_counts()

sample_label
train    17004
test      7288
Name: count, dtype: int64

In [111]:
pd.crosstab(df_pred["sample_label"], df_pred["y"], margins=True)

y,0.0,1.0,All
sample_label,,,
test,7115,173,7288
train,16601,403,17004
All,23716,576,24292


In [112]:
df_pred_fill=get_fill_df(df_pred)
df_pred_fill

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,settlement_status,settlement_amount,settlement_percentage,settlement_term,y,sample_label
0,6000.0,6000.0,6000.0,36.0,12.39,200.41,2,0.016729,0.000000,10.00000,0.021291,66000.0,0.021171,0.023298,0.024599,0.025316,26.49,1.0,675.0,679.0,0.0,5.0000,70.61489,9.0,0.0,23576.0,93.2,32.0,0.011226,0.0,0.0,7134.314182,7134.31,6000.0,1134.31,0.0,0.0,0.0,1931.91,649.0,645.0,0.0,47.000000,1.0,0.021163,118760.241509,19.244828,0.155088,0.0,0.0,494727.0,1.004024,2.763644,0.750026,1.691943,19.54036,36577.834932,70.829963,1.344166,2.876199,5785.482616,57.288868,25300.0,1.044465,1.584649,2.229857,6.0,54970.0,57.0,99.6,0.0,0.0,106.000000,182.0,9.0,7.0,4.0,13.0,39.473684,7.000000,5.000000,1.0,3.0,5.0,3.0,13.0,8.0,5.0,20.0,5.0,9.0,0.0,0.0,0.0,2.0,93.7,100.0,0.0,0.0,542288.0,45904.0,16200.0,32687.0,35042.396396,670.285285,674.285285,0.675676,1.75976,11.384384,55.679692,2.828829,12.987988,0.039039,0.066066,37.675439,0.023183,0.44186,0.430345,0.40875,3.0,158.243151,3.0,12.09589,0.416454,468.269571,12177.284658,205.683562,0.023406,0.022838,1.0,4330.473684,53.421579,14.526316,0.0,train
1,6000.0,6000.0,6000.0,36.0,19.05,220.09,3,0.016129,0.023815,10.00000,0.021291,69680.0,0.021171,0.023298,0.024599,0.021951,13.28,0.0,665.0,669.0,0.0,34.6904,70.61489,24.0,0.0,13389.0,88.1,37.0,0.011226,0.0,0.0,7438.338303,7438.34,6000.0,1438.34,0.0,0.0,0.0,3483.51,659.0,655.0,0.0,43.867711,1.0,0.021163,118760.241509,19.244828,0.155088,0.0,0.0,180964.0,1.004024,2.763644,0.750026,1.691943,19.54036,36577.834932,70.829963,1.344166,2.876199,5785.482616,57.288868,15200.0,1.044465,1.584649,2.229857,6.0,7540.0,1112.0,91.2,0.0,0.0,108.000000,99.0,9.0,3.0,3.0,26.0,39.473684,7.000000,36.005132,0.0,6.0,8.0,6.0,7.0,24.0,8.0,9.0,8.0,24.0,0.0,0.0,0.0,4.0,100.0,83.3,0.0,0.0,185279.0,77500.0,12700.0,62287.0,35042.396396,670.285285,674.285285,0.675676,1.75976,11.384384,55.679692,2.828829,12.987988,0.039039,0.066066,37.675439,0.023

In [113]:
fea_list=list(df_pred.drop(['y','sample_label'], axis=1).columns)

len(fea_list)

133

In [114]:
df_pred.isnull().sum()

loan_amnt                    0
funded_amnt                  0
funded_amnt_inv              0
term                         0
int_rate                     0
                         ...  
settlement_amount        24273
settlement_percentage    24273
settlement_term          24273
y                            0
sample_label                 0
Length: 135, dtype: int64

### 变量统计

#### iv_miss

In [115]:
def iv_miss(df, var_list, y):
    
    df_tmp=df[df[y].notnull()].copy()
    
    iv_all=toad.quality(df_tmp[var_list+[y]], 
                        target=y, 
                        indicators = ['iv','unique'])[['unique','iv']]

    miss_per=pd.DataFrame(df[var_list].isnull().sum()/(df.shape[0]))
    miss_per.columns=['缺失率']
    
    result=pd.concat([miss_per, iv_all],axis=1)
    
    return result.sort_values('iv', ascending=False)

# iv 计算
df_iv=iv_miss(df_pred, fea_list, 'y') 
df_iv

,缺失率,unique,iv
out_prncp_inv,0.00000,575.0,16.399329
out_prncp,0.00000,575.0,16.399329
emp_title,0.21044,51.0,3.427609
last_pymnt_amnt,0.00000,22891.0,2.757245
total_rec_prncp,0.00000,1720.0,2.346741
...,...,...,...
inq_last_6mths,0.00000,8.0,0.010395
verification_status,0.00000,3.0,0.008216
policy_code,0.00000,1.0,0.000000
collection_recovery_fee,0.00000,1.0,0.000000


In [116]:
y_name='y'

# 变量筛选

train_selected, drop_col=toad.selection.select(df_pred[fea_list+[y_name]],
                                              y_name,
                                              empty=0.9,
                                              iv=0.02,
                                              corr=0.7,
                                              return_drop=True)
train_selected

,term,grade,sub_grade,emp_title,home_ownership,annual_inc,purpose,addr_state,dti,fico_range_low,mths_since_last_delinq,mths_since_last_record,pub_rec,initial_list_status,out_prncp,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,last_fico_range_high,mths_since_last_major_derog,application_type,tot_coll_amt,open_acc_6m,open_act_il,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_rev_tl,num_bc_sats,num_il_tl,num_rev_accts,num_sats,num_tl_90g_dpd_24m,pct_tl_nvr_dlq,pub_rec_bankruptcies,hardship_flag,disbursement_method,debt_settlement_flag,y
0,36.0,2,0.016729,0.000000,0.021291,66000.0,0.024599,0.025316,26.49,675.0,5.0,NaN,0.0,0.011226,0.0,6000.0,1134.31,0.0,1931.91,649.0,47.0,0.021163,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25300.0,NaN,NaN,NaN,6.0,54970.0,57.0,99.6,0.0,106.0,182.0,9.0,7.0,4.0,13.0,7.0,1.0,5.0,3.0,8.0,20.0,9.0,0.0,93.7,0.0,0.023183,0.023406,0.022838,0.0
1,36.0,3,0.016129,NaN,0.021291,69680.0,0.024599,0.021951,13.28,665.0,NaN,NaN,0.0,0.011226,0.0,6000.0,1438.34,0.0,3483.51,659.0,NaN,0.021163,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15200.0,NaN,NaN,NaN,6.0,7540.0,1112.0,91.2,0.0,108.0,99.0,9.0,3.0,3.0,26.0,7.0,0.0,8.0,6.0,24.0,9.0,24.0,0.0,100.0,0.0,0.023183,0.023406,0.022838,0.0
2,60.0,2,0.042998,0.000000,0.025533,52000.0,0.024599,0.015625,17.38,660.0,45.0,NaN,0.0,0.032699,0.0,20000.0,3298.24,0.0,17162.39,719.0,59.0,0.021163,7506.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12100.0,NaN,NaN,NaN,5.0,7848.0,4594.0,60.1,0.0,157.0,152.0,12.0,12.0,2.0,12.0,13.0,4.0,3.0,3.0,7.0,21.0,6.0,0.0,78.6,0.0,0.023183,0.023406,0.022838,0.0
3,60.0,1,0.017979,0.028571,0.021291,52000.0,0.024599,0.024291,23.63,765.0,NaN,NaN,0.0,0.032699,0.0,16000.0,5441.40,18.2,5426.72,779.0,NaN,0.021163,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20100.0,NaN,NaN,NaN,6.0,12347.0,12290.0,32.1,0.0,109.0,97.0,6.0,6.0,1.0,6.0,13.0,0.0,4.0,5.0,15.0,9.0,16.0,0.0,100.0,0.0,0.023183,0.023406,0.022838,0.0
4,36.0,0,0.010084,NaN,0.021291,220000.0,0.024599,0.029680,7.91,730.0,NaN,NaN,0.0,0.011226,0.0,25000.0,2579.97,0.0,765.77,759.0,NaN,0.021163,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86000.0,NaN,NaN,NaN,1.0,141185.0,8447.0,61.4,0.0,132.0,307.0,48.0,5.0,6.0,110.0,6.0,0.0,3.0,2.0,5.0,7.0,7.0,0.0,100.0,0.0,0.023183,0.023406,0.022838,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24287,36.0,1,0.015302,0.043478,0.021291,52000.0,0.020298,0.030261,13.52,785.0,NaN,NaN,0.0,0.011226,0.0,16000.0,2493.86,0.0,3541.82,749.0,NaN,0.021163,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80500.0,NaN,NaN,NaN,3.0,6062.0,40564.0,24.2,0.0,135.0,409.0,1.0,1.0,3.0,4.0,4.0,0.0,7.0,6.0,6.0,29.0,18.0,0.0,100.0,0.0,0.023183,0.023406,0.022838,0.0
24288,36.0,3,0.027027,0.000000,0.028267,30000.0,0.020298,0.020000,29.20,685.0,NaN,NaN,0.0,0.032699,0.0,11000.0,2712.39,0.0,5772.23,779.0,NaN,0.021163,0.0,0.0,1.0,1.0,16.0,19871.0,83.0,0.0,3769.0,75.0,17400.0,2.0,0.0,0.0,2.0,5187.0,2420.0,78.6,0.0,16.0,63.0,13.0,13.0,0.0,13.0,16.0,0.0,5.0,3.0,1.0,6.0,6.0,0.0,100.0,0.0,0.023183,0.023406,0.022838,0.0
24289,60.0,3,0.062706,NaN,0.021291,40000.0,0.024599,0.044610,55.63,690.0,51.0,NaN,0.0,0.011226,0.0,33000.0,1604.87,0.0,33727.84,689.0,51.0,0.154321,0.0,5.0,12.0,2.0,6.0,78291.0,74.0,5.0,5069.0,56.0,48700.0,2.0,5.0,6.0,10.0,11616.0,9316.0,59.8,0.0,150.0,146.0,0.0,0.0,1.0,0.0,0.0,4.0,9.0,6.0,33.0,22.0,26.0,0.0,92.9,0.0,0.023183,0.023406,0.022838,0.0
24290,36.0,2,0.024691,0.000000,0.021291,43000.0,0.020690,0.020000,8.48,705.0,NaN,NaN,0.0,0.032699,0.0,3025.0,592.60,0.0,1252.97,694.0,NaN,0.021163,0.0,0.0,0.0

In [117]:
# 剩余变量列表

fea_list_select=list(train_selected.drop('y', axis=1).columns)

print(len(fea_list), len(fea_list_select))

133 60


In [118]:
train_selected.isnull().sum()

term                       0
grade                      0
sub_grade                  0
emp_title               5112
home_ownership             0
                        ... 
pub_rec_bankruptcies      13
hardship_flag              0
disbursement_method        0
debt_settlement_flag       0
y                          0
Length: 61, dtype: int64

### 建模

#### 建模函数

In [119]:
def init_params(model_select='lgb'):  # 初始化参数
    
    params_lr={
        'n_jobs':4,
        'max_iter':100,
        'penalty':'l2',
        'solver':'lbfgs',
        'random_state':1,
        'tol':0.0001,
        'C':1,
    }
    
    params_rf={
        'n_jobs':4,
        'max_samples':0.8,
        'n_estimators':500,
        'max_depth':3,
        'min_samples_leaf':50,
        'max_features':'auto',
        'min_impurity_decrease':0,
        'bootstrap':True,
        'oob_score':True,
        'random_state':1,
    }
    
    params_xgb={
        'objective':'binary:logistic',
        'eval_metric':'auc',
        'nthread':4,
        'n_estimators':500,
        'eta':0.02,
        'max_depth':3,
        'min_child_weight':50,
        'scale_pos_weight':1,
        'gamma':10,
        'reg_alpha':2,
        'reg_lambda':2,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'seed':123
    }
    
    params_lgb={
        'boosting_type': 'gbdt', 
        'objective': 'binary',
        'metric':'auc',
        'n_jobs': 4,
        'n_estimators':params_xgb.get('n_estimators',100),
        'learning_rate': params_xgb.get('eta',0.1), 
        'max_depth':params_xgb.get('max_depth',4), 
        'num_leaves': params_xgb.get('num_leaves',20), 
        'max_bin':255,
        'subsample_for_bin':100000, # 构建直方图的样本量
        'min_split_gain':params_xgb.get('gamma',10),
        'min_child_samples':params_xgb.get('min_child_weight',50),
        'colsample_bytree': params_xgb.get('colsample_bytree',0.8),
        'subsample': params_xgb.get('subsample',0.8), 
        'subsample_freq': 1,   # 每 k 次迭代执行bagging      
        'feature_fraction_seed':2,
        'bagging_seed': 1,
        'reg_alpha':params_xgb.get('reg_alpha',10),
        'reg_lambda':params_xgb.get('reg_lambda',10),
        'scale_pos_weight':params_xgb.get('scale_pos_weight',1), # 等价于is_unbalance=False
        'silent':True,
        'random_state':1,
     }
    
    if model_select=='lr':
        return params_lr
    if model_select=='rf':
        return params_rf
    if model_select=='xgb':
        return params_xgb
    if model_select=='lgb':
        return params_lgb

def ks_auc_value(y_value,df,model): # 计算 ks、auc
    y_pred=model.predict_proba(df)[:,1]
    fpr,tpr,thresholds= roc_curve(list(y_value),list(y_pred))
    ks=max(tpr-fpr)
    auc= roc_auc_score(list(y_value),list(y_pred))
    return ks,auc

def model_train_sklearn(df,y_name,model_fea,model_select='lgb'):
    
    params=init_params(model_select=model_select)
    
    if model_select=='xgb':
        model=XGBClassifier(**params)
    elif model_select=='lgb':
        model=lgb.LGBMClassifier(**params)
    elif model_select=='rf':
        model=RandomForestClassifier(**params)
    elif model_select=='lr':
        model=LogisticRegression(**params)
        
    x_train,y_train=df[df.sample_label=='train'][model_fea], df[df.sample_label=='train'][y_name]
    
    x_test,y_test=df[df.sample_label=='test'][model_fea], df[df.sample_label=='test'][y_name]
    
    if model_select in ['rf','lr']:
        model.fit(x_train, y_train)
    else:
        model.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_test, y_test)])
    
    train_ks, train_auc=ks_auc_value(y_train,x_train,model)
    test_ks, test_auc=ks_auc_value(y_test,x_test,model)
    
    model_result={
        'train_good':(y_train.count()-y_train.sum()),'train_bad':y_train.sum(),
        'test_good':(y_test.count()-y_test.sum()),'test_bad':y_test.sum(),
        'train_ks':train_ks,'train_auc':train_auc,
        'test_ks':test_ks,'test_auc':test_auc,
    }
    
    return model_result,model

In [120]:
xgb_model_result, xgb_model=model_train_sklearn(df_pred,'y',fea_list_select,model_select='xgb')

lgb_model_result, lgb_model=model_train_sklearn(df_pred,'y',fea_list_select,model_select='lgb')

rf_model_result, rf_model=model_train_sklearn(df_pred_fill,'y',fea_list_select,model_select='rf')

lr_model_result, lr_model=model_train_sklearn(df_pred_fill,'y',fea_list_select,model_select='lr')

[0]	validation_0-auc:0.94672	validation_1-auc:0.59860
[1]	validation_0-auc:0.98245	validation_1-auc:0.89176
[2]	validation_0-auc:0.98779	validation_1-auc:0.93852
[3]	validation_0-auc:0.98831	validation_1-auc:0.92519
[4]	validation_0-auc:0.98813	validation_1-auc:0.93279
[5]	validation_0-auc:0.98897	validation_1-auc:0.92541
[6]	validation_0-auc:0.98961	validation_1-auc:0.92171
[7]	validation_0-auc:0.98985	validation_1-auc:0.90807
[8]	validation_0-auc:0.98967	validation_1-auc:0.90984
[9]	validation_0-auc:0.98954	validation_1-auc:0.90442
[10]	validation_0-auc:0.98941	validation_1-auc:0.89766
[11]	validation_0-auc:0.98898	validation_1-auc:0.88878
[12]	validation_0-auc:0.98872	validation_1-auc:0.88334
[13]	validation_0-auc:0.98857	validation_1-auc:0.87295
[14]	validation_0-auc:0.98880	validation_1-auc:0.88747
[15]	validation_0-auc:0.98899	validation_1-auc:0.87409
[16]	validation_0-auc:0.99395	validation_1-auc:0.95530
[17]	validation_0-auc:0.99540	validation_1-auc:0.97684
[18]	validation_0-au

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [121]:
pd.DataFrame([xgb_model_result,
              lgb_model_result,
              rf_model_result,
              lr_model_result],
             index=['xgb','lgb','rf','lr'])

,train_good,train_bad,test_good,test_bad,train_ks,train_auc,test_ks,test_auc
xgb,16601.0,403.0,7115.0,173.0,0.995037,0.999948,1.000000,1.000000
lgb,16601.0,403.0,7115.0,173.0,0.997519,0.999959,1.000000,1.000000
rf,16601.0,403.0,7115.0,173.0,0.997711,0.999993,0.999859,0.999990
lr,16601.0,403.0,7115.0,173.0,0.994796,0.997172,0.994220,0.995729
